In [125]:
import numpy as np 
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchsummary import summary
import torch.nn.functional as F
from torchviz import make_dot
from torch.autograd import Variable

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [3]:
class same_padding(nn.Module):
    def __init__(self, kernel_size) -> None:
        super(same_padding, self).__init__()
        self.pad_size = int(kernel_size/2 - 1)

    def forward(self, input):
        return F.pad(input, (0, 0, self.pad_size, self.pad_size))

In [4]:
class MTEX_CNN(nn.Module):
    def __init__(self, input_shape, n_class):
        super(MTEX_CNN, self).__init__()
        
        self.time_step = input_shape[0]
        self.features_size = input_shape[1]
        
        self.global_features_learning= nn.Sequential(
            
            same_padding(kernel_size = 8),
            nn.Conv2d(in_channels = 1, out_channels = 64, kernel_size = (8,1), stride = (2,1)),
            nn.ReLU(),
            nn.Dropout(p=0.4),
            
            same_padding(kernel_size = 6),
            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = (6,1), stride = (2,1)),
            nn.ReLU(),
            nn.Dropout(p=0.4),
            
            nn.Conv2d(in_channels = 128, out_channels = 1, kernel_size = (1,1), stride=(1, 1), padding = 'same'),
            nn.ReLU(),
        )
            
        self.time_series_learning = nn.Sequential(
            nn.Conv1d(in_channels = self.features_size, out_channels = 128, 
                     kernel_size = 4, stride = 2, padding = 'valid'),
            nn.ReLU(),
            nn.Dropout(p = 0.4)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(14*128, 128), 
            nn.ReLU(),
            nn.Linear(in_features = 128, out_features = n_class),
            nn.Softmax(dim=1)
        )
    
    def forward(self, x):
        x = self.global_features_learning(x)
        x = torch.reshape(x, (-1, self.features_size, int(self.time_step / 4)))
        x = self.time_series_learning(x)
        x = torch.reshape(x, (-1, 14*128))
        x = self.classifier(x)

        return x

In [5]:
mod = MTEX_CNN(input_shape=(120, 44), n_class=2).to(device)
MTEX_CNN_summary = summary(mod, [1, 120, 44])

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 1, 30, 44]           --
|    └─same_padding: 2-1                 [-1, 1, 126, 44]          --
|    └─Conv2d: 2-2                       [-1, 64, 60, 44]          576
|    └─ReLU: 2-3                         [-1, 64, 60, 44]          --
|    └─Dropout: 2-4                      [-1, 64, 60, 44]          --
|    └─same_padding: 2-5                 [-1, 64, 64, 44]          --
|    └─Conv2d: 2-6                       [-1, 128, 30, 44]         49,280
|    └─ReLU: 2-7                         [-1, 128, 30, 44]         --
|    └─Dropout: 2-8                      [-1, 128, 30, 44]         --
|    └─Conv2d: 2-9                       [-1, 1, 30, 44]           129
|    └─ReLU: 2-10                        [-1, 1, 30, 44]           --
├─Sequential: 1-2                        [-1, 128, 14]             --
|    └─Conv1d: 2-11                      [-1, 128, 14]             22,656
|    

In [6]:
model = MTEX_CNN(input_shape=(120, 44), n_class=2)
x = torch.randn([1, 120, 44])
y = model(x)
make_dot(y.mean(), params=dict(model.named_parameters()), show_attrs=True, show_saved=True)

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [7]:
MTEX_CNN(input_shape=(120, 44), n_class=2)

MTEX_CNN(
  (global_features_learning): Sequential(
    (0): same_padding()
    (1): Conv2d(1, 64, kernel_size=(8, 1), stride=(2, 1))
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): same_padding()
    (5): Conv2d(64, 128, kernel_size=(6, 1), stride=(2, 1))
    (6): ReLU()
    (7): Dropout(p=0.4, inplace=False)
    (8): Conv2d(128, 1, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (9): ReLU()
  )
  (time_series_learning): Sequential(
    (0): Conv1d(44, 128, kernel_size=(4,), stride=(2,), padding=valid)
    (1): ReLU()
    (2): Dropout(p=0.4, inplace=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=1792, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=2, bias=True)
    (3): Softmax(dim=1)
  )
)

In [100]:
class XCM(nn.Module):
    def __init__(self, input_shape, n_class, window_size=0.2, filters_num = 128):
        super(XCM, self).__init__()
        
        self.time_step = input_shape[0]
        self.features_size = input_shape[1]
        self.filters_num = filters_num
        
        self.global_features_learning= nn.Sequential(
            
            nn.Conv2d(in_channels = 1, out_channels = self.filters_num, 
                      kernel_size = (int(window_size*self.time_step) , 1) ,stride = (1,1), padding='same'),
            nn.BatchNorm2d(self.filters_num),
            nn.ReLU(),
            
            #nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = (6,1), stride = (2,1)),
            #nn.ReLU(),
            #nn.Dropout(p=0.4),
            
            nn.Conv2d(in_channels = self.filters_num, out_channels = 1, kernel_size = (1,1), stride=(1, 1), padding = 'same'),
            nn.ReLU(),
        )
        
        self.temp_feature_learning =  nn.Sequential(
            nn.Conv1d(in_channels = self.features_size, out_channels = self.filters_num, 
                     kernel_size = int(window_size*self.time_step), stride = 1, padding = 'same'),
            nn.BatchNorm1d(self.filters_num),
            nn.ReLU(),
            nn.Conv1d(in_channels =self.filters_num, out_channels = 1, kernel_size = 1,
                      stride=1),
            nn.ReLU(),
        )
        
        self.time_series_learning = nn.Sequential(
            nn.Conv1d(in_channels = self.features_size + 1, out_channels = self.filters_num, 
                     kernel_size = int(window_size*self.time_step), stride = 1, padding = 'same'),
            nn.BatchNorm1d(self.filters_num), 
            nn.ReLU(), 
        )
        
        self.global_avg_pool = nn.AvgPool2d((1, self.time_step))
        
        self.classifier = nn.Sequential(
            nn.Linear(in_features = self.filters_num, out_features = n_class),
            nn.Softmax(dim=1)
        )
        
    def forward(self, x):
        
        temp_x = torch.reshape(x, (-1, self.features_size, self.time_step))
        x = self.global_features_learning(x)
        x = torch.reshape(x, (-1, self.features_size, self.time_step))
        temp_x = self.temp_feature_learning(temp_x)
        z = torch.concat((x, temp_x), dim = 1)
        
        
        z = self.time_series_learning(z)
        z = self.global_avg_pool(z)
        z = torch.reshape(z, (-1, self.filters_num))
        z = self.classifier(z)

        return z
        

In [101]:
mod = XCM(input_shape=(120, 44), n_class=2,  window_size=0.2, filters_num = 128).to(device)
XCM_summary = summary(mod, [1, 120, 44])

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 1, 120, 44]          --
|    └─Conv2d: 2-1                       [-1, 128, 120, 44]        3,200
|    └─BatchNorm2d: 2-2                  [-1, 128, 120, 44]        256
|    └─ReLU: 2-3                         [-1, 128, 120, 44]        --
|    └─Conv2d: 2-4                       [-1, 1, 120, 44]          129
|    └─ReLU: 2-5                         [-1, 1, 120, 44]          --
├─Sequential: 1-2                        [-1, 1, 120]              --
|    └─Conv1d: 2-6                       [-1, 128, 120]            135,296
|    └─BatchNorm1d: 2-7                  [-1, 128, 120]            256
|    └─ReLU: 2-8                         [-1, 128, 120]            --
|    └─Conv1d: 2-9                       [-1, 1, 120]              129
|    └─ReLU: 2-10                        [-1, 1, 120]              --
├─Sequential: 1-3                        [-1, 128, 120]            --
|  

In [102]:
XCM(input_shape=(120, 44), n_class=2,  window_size=0.2, filters_num = 128)

XCM(
  (global_features_learning): Sequential(
    (0): Conv2d(1, 128, kernel_size=(24, 1), stride=(1, 1), padding=same)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(128, 1, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (4): ReLU()
  )
  (temp_feature_learning): Sequential(
    (0): Conv1d(44, 128, kernel_size=(24,), stride=(1,), padding=same)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv1d(128, 1, kernel_size=(1,), stride=(1,))
    (4): ReLU()
  )
  (time_series_learning): Sequential(
    (0): Conv1d(45, 128, kernel_size=(24,), stride=(1,), padding=same)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (global_avg_pool): AvgPool2d(kernel_size=(1, 120), stride=(1, 120), padding=0)
  (classifier): Sequential(
    (0): Linear(in_features=128, out_features=2, bias=True)
 

In [456]:
class Upsampling1D(torch.nn.Module):
    def __init__(self, scale_factor):
        super(Upsampling1D, self).__init__()
        self.upsampling2D = nn.UpsamplingNearest2d(scale_factor=scale_factor)

    def forward(self, x):
        x = torch.unsqueeze(x, 1)
        x = self.upsampling2D(x)
        
        return x

In [468]:
class TSEM(nn.Module):
    def __init__(self, input_shape, n_class, window_size=0.2, filters_num = 128):
        super(TSEM, self).__init__()
        
        self.time_step = input_shape[0]
        self.features_size = input_shape[1]
        self.filters_num = filters_num
        self.hidden = int(window_size*self.time_step)
        
        self.global_features_learning= nn.Sequential(
            
            nn.Conv2d(in_channels = 1, out_channels = self.filters_num, 
                      kernel_size = (int(window_size*self.time_step) , 1) ,stride = (1,1), padding='same'),
            nn.BatchNorm2d(self.filters_num),
            nn.ReLU(),
            
            nn.Conv2d(in_channels = self.filters_num, out_channels = 1, kernel_size = (1,1), stride=(1, 1), padding = 'same'),
            nn.ReLU(),
        )
        
        self.temp_feature_learning =nn.LSTM(input_size = self.features_size, hidden_size = int(window_size*self.time_step), num_layers = 1, batch_first=True)

        
        self.upsampling = Upsampling1D(scale_factor = 5)
        
        self.time_series_learning = nn.Sequential(
            nn.Conv1d(in_channels = self.features_size, out_channels = self.filters_num, 
                     kernel_size = int(window_size*self.time_step), stride = 1, padding = 'same'),
            nn.BatchNorm1d(self.filters_num), 
            nn.ReLU(), 
        )
        
        self.global_avg_pool = nn.AvgPool2d((1, self.time_step))
        
        self.classifier = nn.Sequential(
            nn.Linear(in_features = self.filters_num, out_features = n_class),
            nn.Softmax(dim=1)
        )
        
    def forward(self, x):
        
        temp_x = torch.reshape(x, (-1, self.time_step, self.features_size))
        x = self.global_features_learning(x)
        x = torch.reshape(x, (-1, self.features_size, self.time_step))
        temp_x, (h_out, c_out) = self.temp_feature_learning(temp_x)
        temp_x = h_out[:,-1,:]
        temp_x = self.upsampling(temp_x)
        
        print((x * temp_x).size())
        z = x * temp_x
        z = self.time_series_learning(z)
        z = self.global_avg_pool(z)
        z = torch.reshape(z, (-1, self.filters_num))
        z = self.classifier(z)

        return temp_x
        

In [469]:
mod = TSEM(input_shape=(120, 44), n_class=2,  window_size=0.2, filters_num = 128).to(device)
TSEM_summary = summary(mod, [1, 120, 44])

torch.Size([2, 44, 120])
Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 1, 120, 44]          --
|    └─Conv2d: 2-1                       [-1, 128, 120, 44]        3,200
|    └─BatchNorm2d: 2-2                  [-1, 128, 120, 44]        256
|    └─ReLU: 2-3                         [-1, 128, 120, 44]        --
|    └─Conv2d: 2-4                       [-1, 1, 120, 44]          129
|    └─ReLU: 2-5                         [-1, 1, 120, 44]          --
├─LSTM: 1-2                              [-1, 120, 24]             6,720
├─Upsampling1D: 1-3                      [-1, 1, 120]              --
|    └─UpsamplingNearest2d: 2-6          [-1, 1, 120]              --
├─Sequential: 1-4                        [-1, 128, 120]            --
|    └─Conv1d: 2-7                       [-1, 128, 120]            135,296
|    └─BatchNorm1d: 2-8                  [-1, 128, 120]            256
|    └─ReLU: 2-9                         [-1, 